In [ ]:

#Python 3 notebook
import numpy as np
import pandas as pd
from functools import reduce
from math import floor, ceil


In [ ]:
df = pd.read_csv('data.csv',header = 1, names =['date', 'prices', 'gold', 'compound', 'neg', 'neu', 'pos' ] )
df = df.set_index('date')
#Obtaining the past data in a different column
df['past_prices'] = df.loc[:,'prices'].shift(7)
df = df.iloc[7:df.shape[0]]
#Splitting train and test dataset
train_size = floor(0.8*df.shape[0])
train = df.iloc[0:train_size]
test = df.iloc[train_size:]
print('Total data size: ',df.shape[0],'. Train data size: ', train.shape[0], '. Test data size: ', test.shape[0], '.',sep='')

In [ ]:
#df


In [ ]:
#train


In [ ]:
#test

In [ ]:
def offset_value(test_start_date, test, predictions_df,idx):
    temp_date = test_start_date
    (a,)=idx.shape
    average_last_5_days_test = 0
    average_upcoming_5_days_predicted = 0
    total_days = a
    for i in range(total_days):
        average_last_5_days_test += test.loc[temp_date, 'prices']
        #temp_date = datetime.strptime(temp_date, "%Y-%m-%d").date()
        #difference = datetime.strptime(idx[i], "%Y-%m-%d")
        #temp_date = difference.strftime('%Y-%m-%d')
        temp_date = idx[i]
        #print(difference) 
    average_last_5_days_test = average_last_5_days_test / total_days

    temp_date = test_start_date
    for i in range(total_days):
        average_upcoming_5_days_predicted += predictions_df.loc[temp_date, 'prices']
        #temp_date = datetime.strptime(temp_date, "%Y-%m-%d").date()
        #difference = datetime.strptime(idx[i], "%Y-%m-%d")
        #temp_date = difference.strftime('%Y-%m-%d')
        temp_date = idx[i]
    average_upcoming_5_days_predicted = average_upcoming_5_days_predicted / total_days
    difference_test_predicted_prices = (average_last_5_days_test - average_upcoming_5_days_predicted)
    return difference_test_predicted_prices

In [ ]:
from treeinterpreter import treeinterpreter as ti
from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# average_upcoming_5_days_predicted += predictions_df.loc[temp_date, 'prices']
# # Converting string to date time
# temp_date = datetime.strptime(temp_date, "%Y-%m-%d").date()
# # Adding one day from date time
# difference = temp_date + timedelta(days=1)
# # Converting again date time to string
# temp_date = difference.strftime('%Y-%m-%d')
        
# start_year = datetime.strptime(train_start_date, "%Y-%m-%d").date().month


prediction_list = []

# Splitting the training and testing data
#train_start_date = str(year) + '-01-01'
#train_end_date = str(year) + '-10-31'
#test_start_date = str(year) + '-11-01'
#test_end_date = str(year) + '-12-31'
#train = df.ix[train_start_date : train_end_date]
#test = df.ix[test_start_date:test_end_date]
    
    # Calculating the sentiment score
sentiment_score_list = []
for date, row in train.T.iteritems():
    sentiment_score = np.asarray([train.loc[date, 'compound'],train.loc[date, 'neg'],train.loc[date, 'neu'],train.loc[date, 'pos']])
    #sentiment_score = np.asarray([df.loc[date, 'neg'],df.loc[date, 'pos']])
    sentiment_score_list.append(sentiment_score)
numpy_df_train = np.asarray(sentiment_score_list)
sentiment_score_list = []
for date, row in test.T.iteritems():
    sentiment_score = np.asarray([test.loc[date, 'compound'],test.loc[date, 'neg'],test.loc[date, 'neu'],test.loc[date, 'pos']])
    #sentiment_score = np.asarray([df.loc[date, 'neg'],df.loc[date, 'pos']])
    sentiment_score_list.append(sentiment_score)
numpy_df_test = np.asarray(sentiment_score_list)

 # Generating models
lr = LogisticRegression()
lr.fit(numpy_df_train, train['prices'])

prediction = lr.predict(numpy_df_test)
prediction_list.append(prediction)
#print train_start_date + ' ' + train_end_date + ' ' + test_start_date + ' ' + test_end_date

test_start_date = '2017-01-27'
test_end_date = '2018-01-02'
#train_start_date = '2013-05-13'
#train_end_date = '2017-01-26'
#idx = pd.date_range(test_start_date, test_end_date)
idx = test.index
#print year
predictions_df_list = pd.DataFrame(data=prediction[0:], index = idx, columns=['prices'])
    
difference_test_predicted_prices = offset_value(test_start_date, test, predictions_df_list,idx)
# Adding offset to all the advpredictions_df price values
predictions_df_list['prices'] = predictions_df_list['prices'] + difference_test_predicted_prices

#predictions_df_list['actual_value'] = test['prices']
#predictions_df_list.plot()
# Smoothing the plot
predictions_df_list['ewma'] = pd.ewma(predictions_df_list["prices"], span=10)
predictions_df_list['actual_value'] = test['prices']
predictions_df_list['actual_value_ewma'] = pd.ewma(predictions_df_list["actual_value"], span=10)
# Changing column names
predictions_df_list.columns = ['predicted_price', 'average_predicted_price', 'actual_price', 'average_actual_price']
predictions_df_list.plot()
predictions_df_list_average = predictions_df_list[['average_predicted_price', 'average_actual_price']]
predictions_df_list_average.plot()
#plt.plot( range(0,test.shape[0]), predictions_df_list, range(0,test.shape[0]), test_data.loc[:,'prices'].as_matrix())
#predictions_df_list.show()
plt.show()




    


In [ ]:
lr.classes_

In [ ]:
lr.coef_[0]

In [ ]:
from sklearn import metrics
import numpy as np


# calculate MAE using scikit-learn
mae=metrics.mean_absolute_error(test['prices'], predictions_df_list['predicted_price'])
print('MAE:',mae)

